In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib tk

import os 
import time 
import pianoq
import datetime
from TC300_COMMAND_LIB import TC300
from pianoq.lab.asi_cam import ASICam
from pianoq.lab.photon_counter import PhotonCounter
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.photon_scan import PhotonScanner

can't import TimeTagger


In [ ]:
tc = TC300()
serial = tc.list_devices()[0][0]
tc.open(serial, 115200, 3)
tc.set_target_temperature(2, 75)

In [ ]:
tc.close()

In [ ]:
cam = ASICam()
base_path = r'G:\My Drive\People\Ronen\PHD\SPDC2025\phase_matching'
cam.set_exposure(1)

In [ ]:
cam.close()

## Find phase matching temperature

In [ ]:
t = 0
while t < 72:
    time.sleep(2)
    tt = [0]
    tc.get_actual_temperature(2, tt)
    t = round(tt[0], 4)
    cam.save_image(os.path.join(base_path, f"T={t}.fits"))

## Initial single counts after MMF

In [3]:
cam = ASICam()

In [ ]:
cam.set_gain(0)
cam.set_binning(1)
cam.set_exposure(5)
cam.set_roi(2500, 1400, 600, 600)

In [12]:
cam.show_image()
cam.save_image(r'G:\My Drive\People\Ronen\PHD\SPDC2025\After MMF\no_pump_lens')

## Check photon counter 

In [21]:
ph = PhotonCounter(serial_port='COM8')

In [23]:
ph.close()

In [22]:
ph.read()

(array([0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0.]),
 1.0)

In [ ]:
while True:
    x = ph.read()
    print(x[0])

In [16]:
ph.close()

## Photon scan 

In [2]:
timeout = 0 
x_motor = ThorlabsKcubeDC(serial_number=27253522, timeout=timeout)
y_motor = ThorlabsKcubeStepper(serial_number=26001271, timeout=timeout)
ph = PhotonCounter(serial_port='COM8')

In [3]:
x_motor.move_relative(0.1)
y_motor.move_relative(-0.1)

In [10]:
x_motor.timeout = 50000
y_motor.timeout = 50000

In [5]:
print(x_motor.get_position())
print(y_motor.get_position())

15.5997
13.999997


In [ ]:
integration_time = 1 
start_x = 15 # 16.5
start_y = 13.25 # 14.75  
x_pixels = 30
y_pixels = 30
pixel_size_x = 0.05
pixel_size_y = 0.05
# name = 'find_image'
name = 'first_full_dx=0.05_with_lens'

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
dir_path = r'G:\My Drive\People\Ronen\PHD\SPDC2025\phase_matching\Scans'

path = f'{dir_path}\\{timestamp}_{name}.scan'
scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y,
                        run_name=name, is_timetagger=False, coin_window=2e-9, saveto_path=path)

In [ ]:
single1s, single2s, coincidences = scanner.scan(x_motor=x_motor, y_motor=y_motor, ph=ph)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 11493. Singles2: 9191. Coincidence: 0.00.
dur: 2. pix: 0, 1. Singles1: 11241. Singles2: 9032. Coincidence: 0.00.
dur: 4. pix: 0, 2. Singles1: 10678. Singles2: 8792. Coincidence: 1.00.
dur: 5. pix: 0, 3. Singles1: 11622. Singles2: 9321. Coincidence: 1.00.
dur: 7. pix: 0, 4. Singles1: 11027. Singles2: 8966. Coincidence: 0.00.
dur: 9. pix: 0, 5. Singles1: 11140. Singles2: 8981. Coincidence: 0.00.
dur: 11. pix: 0, 6. Singles1: 6198. Singles2: 4839. Coincidence: 2.00.
dur: 13. pix: 0, 7. Singles1: 6091. Singles2: 4927. Coincidence: 1.00.
dur: 14. pix: 0, 8. Singles1: 6174. Singles2: 4850. Coincidence: 0.00.
dur: 16. pix: 0, 9. Singles1: 6271. Singles2: 4892. Coincidence: 0.00.
dur: 18. pix: 0, 10. Singles1: 6242. Singles2: 4939. Coincidence: 1.00.
dur: 20. pix: 0, 11. Singles1: 6295. Singles2: 5013. Coincidence: 0.00.
dur: 22. pix: 0, 12. Singles1: 6189. Singles2: 4814. Coincidence: 0.00.
dur: 23. pix: 0, 13. Singles1

In [ ]:
# x_motor.close() # pesky bug?
y_motor.close()  
ph.close()